Diabetes and Chronic kidney disease


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
# Load Diabetes Dataset
diabetes_columns = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness",
                    "Insulin", "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"]
diabetes_df = pd.read_csv(diabetes_url, names=diabetes_columns)

# Split features and labels
diabetes_X = diabetes_df.drop(columns=["Outcome"])
diabetes_y = diabetes_df["Outcome"]


NameError: name 'diabetes_url' is not defined

In [ ]:
# Load CKD Dataset
ckd_url = "https://raw.githubusercontent.com/mansoordaku/ckdisease/master/kidney_disease.csv"
ckd_df = pd.read_csv("kidney_disease.csv")

# Drop unnecessary columns
ckd_df = ckd_df.drop(columns=['id'])

# Handle missing values (Replace '?' with NaN and drop rows with NaN)
ckd_df.replace('?', np.nan, inplace=True)
ckd_df.dropna(inplace=True)

# Convert categorical labels to binary (Yes=1, No=0)
label_encoder = LabelEncoder()
ckd_df['classification'] = label_encoder.fit_transform(ckd_df['classification'])  # CKD: 1, Not CKD: 0

# Split features and labels
ckd_X = ckd_df.drop(columns=["classification"])
ckd_y = ckd_df["classification"]


In [ ]:
# Convert categorical columns to numerical values
for col in ckd_X.columns:
    if ckd_X[col].dtype == 'object':  # If column contains text (categorical data)
        ckd_X[col] = LabelEncoder().fit_transform(ckd_X[col])


In [ ]:
# Standardize numerical data
scaler = StandardScaler()

# Normalize Diabetes Data
diabetes_X_scaled = scaler.fit_transform(diabetes_X)

# Normalize CKD Data
ckd_X_scaled = scaler.fit_transform(ckd_X)

# Ensure same number of features (pad or truncate)
num_features = max(diabetes_X_scaled.shape[1], ckd_X_scaled.shape[1])

diabetes_X_scaled = np.pad(diabetes_X_scaled, [(0, 0), (0, num_features - diabetes_X_scaled.shape[1])], mode='constant')
ckd_X_scaled = np.pad(ckd_X_scaled, [(0, 0), (0, num_features - ckd_X_scaled.shape[1])], mode='constant')

# Combine datasets
X = np.vstack((diabetes_X_scaled, ckd_X_scaled))
y = np.column_stack((np.hstack((diabetes_y, np.zeros(len(ckd_y)))),  # Diabetes labels
                     np.hstack((np.zeros(len(diabetes_y)), ckd_y)))) # CKD labels

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


NameError: name 'diabetes_X' is not defined

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.xlabel('Epoch')
  plt.ylabel('Binary crossentropy')
  plt.legend()
  plt.grid(True)
  plt.show()

def plot_accuracy(history):
  plt.plot(history.history['accuracy'], label='accuracy')
  plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.grid(True)
  plt.show()



In [ ]:
# Define a Multi-Output Sequential Model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(num_features,)),  # Input Layer
    keras.layers.Dense(32, activation='relu'),  # Hidden Layer
    keras.layers.Dense(8, activation='relu'),  # Hidden Layer
    keras.layers.Dense(2, activation='sigmoid')  # Output Layer (Diabetes & CKD)
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Model Summary
model.summary()


In [ ]:
from re import VERBOSE
# Train the model
history = model.fit(X_train, y_train, epochs=75, batch_size=16, validation_data=(X_test, y_test))



In [ ]:
# Evaluate on Test Data
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Calculate Accuracy for Each Disease
diabetes_acc = accuracy_score(y_test[:, 0], y_pred[:, 0])
ckd_acc = accuracy_score(y_test[:, 1], y_pred[:, 1])

print(f"Diabetes Prediction Accuracy: {diabetes_acc:.4f}")
print(f"CKD Prediction Accuracy: {ckd_acc:.4f}")


# Save the model
model.save("diabetes_ckd_prediction_model.h5")


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

# Load the trained model
model = load_model("diabetes_ckd_prediction_model.h5")

# Load dataset (for consistent feature scaling)
diabetes_columns = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness",
                    "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"]
ckd_columns = ["bp", "sg", "al", "su", "rbc", "pc", "pcc", "ba", "bgr", "bu", "sc", "sod",
               "pot", "hemo", "pcv", "wc", "rc", "htn", "dm", "cad", "appet", "pe", "ane"]

# --- NEW PATIENT DATA ---
new_patient = {
    "Pregnancies": 0, "Glucose": 137, "BloodPressure": 40, "SkinThickness": 35,
    "Insulin": 168, "BMI": 43.1, "DiabetesPedigreeFunction": 2.288, "Age": 33,
    "bp": 140, "sg": 1.02, "al": 0, "su": 0, "rbc": 1, "pc": 1, "pcc": 0, "ba": 0,
    "bgr": 120, "bu": 40, "sc": 1.3, "sod": 140, "pot": 8, "hemo": 15,
    "pcv": 44, "wc": 8300, "rc": 5.5, "htn": 0, "dm": 0, "cad": 0,
    "appet": 1, "pe": 0, "ane": 0
}

# Convert to DataFrame
new_data_df = pd.DataFrame([new_patient])

# Ensure feature order
all_features = diabetes_columns + ckd_columns
new_data_df = new_data_df[all_features]  # Reorder columns

# Normalize input using StandardScaler (Fit on training data for consistency)
scaler = StandardScaler()
new_data_scaled = scaler.fit_transform(new_data_df)  # Scale features

# Ensure correct feature size
expected_features = model.input_shape[1]
actual_features = new_data_scaled.shape[1]

if actual_features < expected_features:
    # Pad with zeros if features are missing
    new_data_scaled = np.pad(new_data_scaled, [(0, 0), (0, expected_features - actual_features)], mode='constant')
elif actual_features > expected_features:
    # Trim extra features if present
    new_data_scaled = new_data_scaled[:, :expected_features]

# Make predictions
predictions = model.predict(new_data_scaled)

# Convert predictions to binary (0 or 1)
predicted_labels = (predictions > 0.5).astype("int32")

# Output the results
diabetes_result = "YES" if predicted_labels[0][0] == 1 else "NO"
ckd_result = "YES" if predicted_labels[0][1] == 1 else "NO"

print(f"Predicted Diabetes Risk: {diabetes_result}")
print(f"Predicted CKD Risk: {ckd_result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Predicted Diabetes Risk: NO
Predicted CKD Risk: NO


Lung Cancer


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
df = pd.read_csv("survey_lung_cancer.csv")  # Ensure the file exists

# Handle missing values: Fill NaN with the most frequent value (mode)
imputer = SimpleImputer(strategy="most_frequent")
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# Convert categorical labels to binary (Yes -> 1, No -> 0)
df_imputed["LUNG_CANCER"] = df_imputed["LUNG_CANCER"].map({"YES": 1, "NO": 0})

# Separate features (X) and target (y)
X = df_imputed.drop(columns=["LUNG_CANCER"])
y = df_imputed["LUNG_CANCER"]

# Convert categorical features (GENDER) to numeric (0 and 1)
X = pd.get_dummies(X, drop_first=True)

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Build the Neural Network Model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_resampled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_resampled, y_train_resampled, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Plot Accuracy & Loss Curves
plt.figure(figsize=(12, 5))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()

plt.show()

# Save the model
model.save("lung_cancer_prediction_model.h5")



In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
